In [ ]:
# Dependencies
import pandas as pd
from pathlib import Path

# Load data files
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

# Read data files into dataframes
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

# combine dataframes into one
school_data_all = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])

# print results thus far
school_data_all.head()


<font size = "7"> District Summary </font>

In [ ]:
# Total number of schools
school_count = school_data_all['School ID'].nunique()

school_count

In [ ]:
# Total number of students
student_count = school_data_all['Student ID'].count()

student_count

In [ ]:
# Total budget (using only school_data df)
total_budget = school_data['budget'].sum()

total_budget

In [ ]:
# Average math score
avg_math_score = school_data_all['math_score'].mean()

avg_math_score

In [ ]:
# Average reading score
avg_reading_score = school_data_all['reading_score'].mean()

avg_reading_score

In [ ]:
# % of students passing math
passing_math_count = school_data_all[(school_data_all["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = (passing_math_count / float(student_count)) * 100

passing_math_percentage


In [ ]:
# % of students passing reading
passing_reading_count = school_data_all[(school_data_all["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = (passing_reading_count / float(student_count)) * 100

passing_reading_percentage 

In [ ]:
# % of students passing math AND reading
passing_both_count = school_data_all[(school_data_all["math_score"] >= 70) & (school_data_all["reading_score"] >= 70)].count()["student_name"]
passing_both_percentage = (passing_both_count / float(student_count)) * 100

passing_both_percentage


In [172]:
# Create a high-level snapshot of the district's key metrics in a DataFrame

# Create a dictionary with variable names as keys and their formatted values as values
new_district_fields = {
    'Total Schools': [school_count],
    'Total Students': ['{:,}'.format(student_count)],
    'Total Budget': ['${:,.2f}'.format(total_budget)],
    'Average Math Score': [avg_math_score],
    'Average Reading Score': [avg_reading_score],
    '% Passing Math': [passing_math_percentage],
    '% Passing Reading': [passing_reading_percentage],
    '% Passing Both': [passing_both_percentage]
}

# Create the DataFrame
district_summary = pd.DataFrame(new_district_fields)

district_summary




,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


<font size = "7"> School Summary </font>

In [ ]:
# create a list of school types
school_types = school_data.set_index('school_name')['type']

school_types

In [ ]:
# Calculate the total student count per school
per_school_counts = school_data_all.groupby('school_name')['Student ID'].count()

per_school_counts


In [ ]:
# Calculate the total school budget
per_school_budget = school_data.groupby('school_name')['budget'].sum()

# Calculate the per capita spending per school
per_school_capita = per_school_budget / school_student_count

# Create a DataFrame with school budget and per capita spending
budgets_df = pd.DataFrame({
    'Total School Budget': per_school_budget,
    'Per Capita Spending': per_school_capita
})

# Format the columns as dollars
budgets_df['Total School Budget'] = budgets_df['Total School Budget'].apply('${:,.2f}'.format)
budgets_df['Per Capita Spending'] = budgets_df['Per Capita Spending'].apply('${:,.2f}'.format)


budgets_df


In [184]:
# Calculate the average test scores per school
per_school_math = school_data_all.groupby('school_name')['math_score'].mean()
per_school_reading = school_data_all.groupby('school_name')['reading_score'].mean()

scores_df = pd.DataFrame({
    'Per School Math': per_school_math,
    'Per School Reading': per_school_reading 
})

scores_df

,Per School Math,Per School Reading
school_name,,
Bailey High School,77.048432,81.033963
Cabrera High School,83.061895,83.975780
Figueroa High School,76.711767,81.158020
Ford High School,77.102592,80.746258
Griffin High School,83.351499,83.816757
Hernandez High School,77.289752,80.934412
Holden High School,83.803279,83.814988
Huang High School,76.629414,81.182722
Johnson High School,77.072464,80.966394


In [ ]:
# Calculate the number of students per school with math scores of 70 or higher
school_students_passing_math = school_data_all.groupby('school_name').apply(lambda x: (x['math_score'] >= 70).sum())

school_students_passing_math 

In [ ]:
# Calculate the number of students per school with reading scores of 70 or higher
school_students_passing_reading = school_data_all.groupby('school_name').apply(lambda x: (x['reading_score'] >= 70).sum())

school_students_passing_reading

In [ ]:
# calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_all[
    (school_data_all["reading_score"] >= 70) & (school_data_all["math_score"] >= 70)
]

school_students_passing_math_and_reading = students_passing_math_and_reading.groupby("school_name").size()

print(school_students_passing_math_and_reading)


In [ ]:
# calculate the passing rates per school
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

passing_rates_df = pd.DataFrame({
    'Passing Math': per_school_passing_math,
    'Passing Reading': per_school_passing_reading,
    'Overall Passing': overall_passing_rate 
})

passing_rates_df




In [186]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.

# Create the DataFrame
per_school_summary = pd.DataFrame({
    'School Type': school_data.set_index('school_name')['type'],
    'Total Students': per_school_counts,
    'Total School Budget': per_school_budget,
    'Per Student Budget': per_school_capita,
    'Average Math Score': per_school_math,
    'Average Reading Score': per_school_reading,
    '% Passing Math': per_school_passing_math,
    '% Passing Reading': per_school_passing_reading,
    '% Overall Passing': overall_passing_rate
})

per_school_summary



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
